# Exotic Options Payoffs

Exotic options are more complex derivatives than standard European options, featuring unique payoff structures. Below is a brief description of the different exotic options included in the code:

### 1. Power Option Call Payoff
The payoff for a power call option is:
$ \text{Payoff (Power Call)} = \max(S_T^n - K, 0) $

- **Power Option Call**:
   - A **Power Call** option has a payoff of $\max(S_T^n - K, 0)$, where the underlying asset price is raised to a power \($n$\).
   - These options allow greater sensitivity to the underlying asset's price movements, as the payoff is dependent on a power of the asset price.

### 2. Power Option Put Payoff
The payoff for a power put option is:
$ \text{Payoff (Power Put)} = \max(K - S_T^n, 0) $

- **Power Option Call/Put**:
   - A **Power Put** option has a payoff of $\max(K - S_T^n, 0)$, where the underlying asset price is raised to a power \($n$\). 
   - These options allow greater sensitivity to the underlying asset's price movements, as the payoff is dependent on a power of the asset price.

### 3. Asset-or-Nothing Call Payoff
For an asset-or-nothing call, the payoff at maturity is:
$ \text{Payoff (Asset-or-Nothing Call)} = 
\begin{cases} 
S_T & \text{if } S_T \geq K \\
0 & \text{otherwise}
\end{cases} $

- **Asset-or-Nothing Call**:
   - For an **Asset-or-Nothing Call**, if the underlying asset price exceeds the strike price \(K\), the option pays the asset price \(S_T\); otherwise, the payoff is 0. 
   - This is a binary option where the payoff is the asset price itself if the option finishes in-the-money.

### 4. Asset-or-Nothing Put Payoff
For an asset-or-nothing put, the payoff at maturity is:
$ \text{Payoff (Asset-or-Nothing Put)} = 
\begin{cases} 
S_T & \text{if } S_T \leq K \\
0 & \text{otherwise}
\end{cases} $

- **Asset-or-Nothing Put**:
   - For an **Asset-or-Nothing Put**, if the underlying asset price does not exceed the strike price \($K$\), the option pays the asset price \($S_T$\); otherwise, the payoff is 0. 
   - This is a binary option where the payoff is the asset price itself if the option finishes in-the-money.

### 5. Digital Call (Cash-or-Nothing Call) Payoff
The payoff for a digital (cash-or-nothing) call option is:
$ \text{Payoff (Digital Call)} = 
\begin{cases} 
1 & \text{if } S_T \geq K \\
0 & \text{otherwise}
\end{cases} $

- **Digital Call(Cash-or-Nothing Call)**:
   - A **Digital Call** pays 1 unit of cash if the asset price is above the strike at maturity, and 0 otherwise. 
   - These are common in structured products, paying a fixed amount if a certain price condition is met.

### 6. Digital Put (Cash-or-Nothing Put) Payoff
The payoff for a digital (cash-or-nothing) put option is:
$ \text{Payoff (Digital Put)} = 
\begin{cases} 
1 & \text{if } S_T \leq K \\
0 & \text{otherwise}
\end{cases} $

- **Digital Put (Cash-or-Nothing Put)**:
   - A **Digital Put** pays 1 unit of cash if the asset price is below the strike at maturity, and 0 otherwise.  
   - These are common in structured products, paying a fixed amount if a certain price condition is met.

### 7. Capped Call Option Payoff
For a capped call option, the payoff is the minimum between the regular call option payoff and a maximum cap \( $M$ \):
$ \text{Payoff (Capped Call)} = \min(\max(S_T - K, 0), M) $

- **Capped Call Option**:
   - The payoff for a **Capped Call** is the lesser of a regular call option payoff and a maximum cap \($M$\). The formula is $\min(\max(S_T - K, 0), M)$.
   - This limits the maximum payoff, providing risk control for both the buyer and seller.

### 8. Corridor Call Option Payoff
The payoff for a corridor call option is a binary outcome that depends on whether the asset price \( $S_T$ \) lies within a specified range between \( $K_1$ \) and \( $K_2$ \):
$ \text{Payoff (Corridor Call)} = 
\begin{cases} 
1 & \text{if } K_1 \leq S_T \leq K_2 \\
0 & \text{otherwise}
\end{cases} $

- **Corridor Call Option**:
   - The **Corridor Call** option pays 1 if the asset price stays within a specified range \([K_1, K_2]\) at maturity and 0 otherwise.
   - This is useful in scenarios where price stability within a defined range is expected.

In [1]:
import numpy as np
from scipy.stats import norm
from math import pi
%run Europea_Options.ipynb

In [2]:
class Exotics(EuropeanOptions):
    
    def __init__(self,S_t,K,T,sigma,r,D,n):
        
        EuropeanOptions.__init__(self,S_t,K,T,sigma,r,D)
        self.n = n 
        
        self.d_1 = ((np.log(self.S_t / (np.power(self.K,1/self.n)))) + ((self.r - self.D) + ((self.n - 1/2) * np.power(self.sigma,2)) * self.T )) / (self.sigma * np.sqrt(self.T))  
        self.d_2 = self.d_1 - self.n * self.sigma * np.sqrt(self.T)
        
    
    def PowerOptionCall(self):
        
        return (np.power(self.S_t, self.n) * np.exp(- self.n * self.D * self.T) * norm.cdf(self.d_1)) - self.K * np.exp(- self.n * (self.r + ((1/2) * (self.n - 1) * np.power(self.sigma,2))) * self.T) * norm.cdf(self.d_2)
    
    
    def PowerOptionPut(self):
        
        call = self.PowerOptionCall()
        
        
        return call - self.S_t * np.exp(- self.D * self.T) + self.K * np.exp(-self.r * self.T)
    
    
    
    def Asset_Or_Nothing_Call(self):
        
        return self.S_t * np.exp(-self.D * self.T) * norm.cdf(self.d1)
    
    
    
    def Asset_Or_Nothing_Put(self):
        
        return self.S_t * np.exp(-self.D * self.T) * norm.cdf(-self.d1)
    
    
    
    def Digital_Call(self):
        
        return np.exp(-self.r * self.T) * norm.cdf(self.d2)
    
    
    def Digital_Put(self):
        
        return np.exp(-self.r * self.T) * norm.cdf(-self.d2) 
    
    
    def Capped_Call_Option(self, M):
        
        Call_bs_1 = self.Call_Black_Scholes()
        self.K = self.K + M 
        Call_bs_2 = self.Call_Black_Scholes()
        
        return Call_bs_1 - Call_bs_2
    
    
    def Corridor_Call_Option(self, K1, K2):
        
        self.K = K1
        Digit_Call_1 = self.Digital_Call()
        self.K = K2 
        Digit_Call_2 = self.Digital_Call()
        
        return Digit_Call_1 - Digit_Call_2 
    
    
    
    
    def shoot_display(self):
        
        import pandas as pd
        
        return pd.DataFrame(data={'Valeur_PowerOptionCall':[self.PowerOptionCall()],
                                 'Valeur_PowerOptionPut':[self.PowerOptionPut()],
                                 'Valeur_Asset_Or_Nothing_Call':[self.Asset_Or_Nothing_Call()],
                                 'Valeur_Asset_Or_Nothing_Put':[self.Asset_Or_Nothing_Put()],
                                 'Valeur_Digital_Call':[self.Digital_Call()],
                                 'Valeur_Digital_Put':[self.Digital_Put()],
                                 'Valeur_Capped_Call_Option':[self.Capped_Call_Option(45)],
                                 'Valeur_Corridor_Call_Option':[self.Corridor_Call_Option(75,95)]}).T.rename(columns={0:'Valeurs'})

In [3]:
option = Exotics(100,75,1,0.3,0.2,0.1,2)

In [4]:
option.shoot_display()

,Valeurs
Valeur_PowerOptionCall,8141.360551
Valeur_PowerOptionPut,8112.281616
Valeur_Asset_Or_Nothing_Call,83.732514
Valeur_Asset_Or_Nothing_Put,6.751228
Valeur_Digital_Call,0.715022
Valeur_Digital_Put,0.103709
Valeur_Capped_Call_Option,32.175990
Valeur_Corridor_Call_Option,0.000000
